This notebook guides you through an exercise in star/galaxy/quasar classification, using k Nearest Neighbors and Decision Trees algorithm.

Written with ❤️ by Viviana Acquaviva for the LSST Data Science Fellowship lecture series.

License: [BSD-3-clause](https://opensource.org/license/bsd-3-clause/)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import StandardScaler
from sklearn import metrics 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict, cross_validate
from sklearn.model_selection import KFold, StratifiedKFold

We can read the data here. They were taken by the Sloan Digital Sky Survey and come from [Kaggle](https://www.kaggle.com/datasets/divyansh22/dummy-astronomy-data).

In [ ]:
data = pd.read_csv('star_classification.csv', sep = ',')

df = data.sample(frac = 0.2, random_state = 11)

df.reset_index(drop = True, inplace=True)

Let's learn a few things about this data frame.

In [ ]:
df.columns

We can get some basic info about the data (numerical columns only!) like this.

In [ ]:
df.describe()

What should be our features and targets?

We can select the following columns:

In [ ]:
seldf = df[['u','g','r','i','z']]

### Feature exploration.

What should we do?

In [ ]:
for col in seldf.columns:
    plt.hist(seldf[col], alpha = 0.5)

When a histogram looks like this, what does it mean?

In [ ]:
for col in seldf.columns:
    plt.hist(seldf[col], alpha = 0.5, bins=100, range = (0,30))

In [ ]:
for col in seldf.columns:
    print(len(np.where(seldf[col] < 0)[0]))

One possible (less than ideal) selection:

In [ ]:
for col in seldf.columns:
    seldf = seldf[seldf[col] >= 0]

In [ ]:
seldf.shape 

In [ ]:
for col in seldf.columns:
    plt.hist(seldf[col], alpha = 0.5, bins=100)

Let's now turn to the target column.

In [ ]:
np.unique(df['class'])

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() #turns categorical into 1 ... N

In [ ]:
y = le.fit_transform(df['class'].values)

In [ ]:
le.classes_

It is also useful to know which class was mapped to which number!

In [ ]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

In [ ]:
np.unique(y)

We can now plot the class distribution to see if something stands out.

In [ ]:
plt.hist(y);

Verdict: Is there anything we should keep in mind?

In [ ]:
target = y[seldf.index]

Now that we have our feature matrix and target vector, we could try and build a model, or try to get some more intel on the relationship between features and target. One way is to build a correlation matrix (easiest with seaborn); another one that I like when there are very few features like in this case, is to make scatter plots of the features and color the points by the label, to see if any obvious gradient shows up. 

In [ ]:
len(seldf.columns)

In [ ]:
#It will take a little time.

fig, axes = plt.subplots(ncols = len(seldf.columns), nrows = len(seldf.columns), figsize=(50,50))

for i, col in enumerate(seldf.columns):
    for j, col2 in enumerate(seldf.columns):
        axes[i,j].scatter(seldf[col], seldf[col2], c = target)
        axes[i,j].set_xlabel(str(seldf[col].name), fontsize = 35)
        axes[i,j].set_ylabel(str(seldf[col2].name), fontsize = 35)

Q: Is there anything special that we can learn from these plots? Do we expect our features to have enough discriminating power? Why or why not?








### Building models

Alright, time to build our first model - let's start with kNN.

This line helps visualize the parameters of any algorithm.

In [ ]:
model.get_params()

We can set up a cross validation strategy like this.

In [ ]:
cv = KFold(n_splits=5, random_state=10, shuffle= True)

This line does the cross validation for us by randomizing the data, creating five folds, building 5 models with the 5 possible training sets, applying each model to the objects in the test fold, and saving the 5 train and test scores in a dictionary.

In [ ]:
scores = cross_validate(model, seldf, target, cv = cv, return_train_score = True)

In [ ]:
print("Time: %.3fs, Mean train score: %.3f, Mean test score: %.3f, std: %.3f"%(scores['score_time'].mean(), scores['train_score'].mean(), scores['test_score'].mean(), scores['test_score'].std()))

Let us now set up a decision tree classifier with default parameters:

In [ ]:
model = DecisionTreeClassifier()

In [ ]:
model.get_params()

Decision trees are more complex than kNN and have a few more hyperparameters to play with. They include: 
- the minimum number of instances in a leaf node; 
- the minimum number of instances required in a split node;
- the maximum depth of tree;
- the minimum information gain required to decide whether a split is "worth it".

In [ ]:
scores = cross_validate(model, seldf, target, cv = cv, return_train_score=True)

In [ ]:
print("Time: %.3fs, Mean train score: %.3f, Mean test score: %.3f, std: %.3f"%(scores['score_time'].mean(), scores['train_score'].mean(), scores['test_score'].mean(), scores['test_score'].std()))

Before sending you on your way with optimizing these models, let's also talk about the great function cross_val_predict. It is used to generate test predictions in the cross validation framework. Just like cross_validate, this line will randomize the data, create five folds (if the random seed is fixed, they will be consistent across our notebooks!), build 5 models with the 5 possible training sets, apply each model to the objects in the test fold, and compile together the predicted values from the 5 test folds. The output will be this vector of predicted values, which has the same size as the target vector.

In [ ]:
ypred = cross_val_predict(model, seldf, target, cv = cv)

Having these predictions is very useful because you can calculate more complex scores, or a confusion matrix (or a scatter plot of true vs predicted values in the case of regression).

In [ ]:
print(metrics.confusion_matrix(target, ypred))

Finally, I'm going to drop here this handy function to calculate and plot learning curves.

Learning curves are helpful in order to visualize the training scores vs the test scores, and how they vary as a function of data set size. They allow us to determine whether we have enough learning data, AND whether we have a high bias or high variance problem.

The source code below is a slight modification of [this code](https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html).

In [ ]:
from sklearn.model_selection import learning_curve

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=5,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring = 'accuracy'):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure(figsize=(10,6))
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel(str(scoring))
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring = scoring)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Test score from cross-validation")

    plt.legend(loc="best")
    return plt

This is a demonstration of how to use the function:

In [ ]:
plot_learning_curve(model, 'DT (default params)', seldf, target,  cv = cv);

In the case, above, would we say that

- We have high bias or high variance? Why?

- We have enough training data or not? Why?

### Tasks

Feel free to do the ones you prefer according to your expertise level and personal preference!

1. Consider (separately) the kNN and DT models with default parameters. Do they suffer from high variance or high bias? Why?

2. Based on your answer above, do you anticipate better results from making the model(s) more complex or simpler? How could this be implemented, in terms of choosing hyperparameters and their range?

3. Play around with hyperparameters for a model of your choice and see what test scores you can get! The highest score wins... a drink of their choice at Monday's dinner?

4. What's something I should have done for kNN and didn't? :) 